# Run some constrained projections

Follow the same setup as in notebook 200, with more scrutiny of outputs including other forcers.

We need to save out the land use forcing in each SSP and ensemble member to feed into the concentration-driven runs (240).

## Basic imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
import xarray as xr
from tqdm import tqdm

from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise
from fair.forcing.ghg import meinshausen2020

## Set up problem

In [ ]:
erf_2co2 = meinshausen2020(
    np.array([554.30, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array([277.15, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array((1.05, 0.86, 1.07)) * np.ones((1, 1, 1, 1)),
    np.ones((1, 1, 1, 3)),
    np.array([True, False, False]),
    np.array([False, True, False]),
    np.array([False, False, True]),
    np.array([False, False, False])
).squeeze()[0]
erf_2co2

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')
df_solar = pd.read_csv('../data/forcing/solar_erf.csv', index_col='year')

In [ ]:
solar_forcing = np.zeros(551)
solar_forcing[:550] = df_solar.loc[1750.5:, 'solar_erf'].values
volcanic_forcing = np.zeros(551)
volcanic_forcing[:270] = df_forc['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
valid_all = np.loadtxt('../data/ar6_ensemble_batches/final_ebm3.csv').astype(np.int64)#[:1000]
valid_all

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')
df_1750co2=pd.read_csv('../data/parameter_sets/co2_concentration_1750.csv')

In [ ]:
species, properties = read_properties()

In [ ]:
seed = 1355763 + 399 * valid_all

In [ ]:
trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
f = FAIR(ch4_method='Thornhill2021')
f.define_time(1750, 2300, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
f.define_species(species, properties)
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

fill(f.forcing, volcanic_forcing[:, None, None] * df_scaling.loc[valid_all, 'Volcanic'].values.squeeze(), specie='Volcanic')
fill(f.forcing, 
     solar_forcing[:, None, None] * 
     df_scaling.loc[valid_all, 'solar_amplitude'].values.squeeze() + 
     trend_shape[:, None, None] * df_scaling.loc[valid_all, 'solar_trend'].values.squeeze(),
     specie='Solar'
)

# climate response
fill(f.climate_configs['ocean_heat_capacity'], df_cr.loc[valid_all, 'c1':'c3'].values)
fill(f.climate_configs['ocean_heat_transfer'], df_cr.loc[valid_all, 'kappa1':'kappa3'].values)
fill(f.climate_configs['deep_ocean_efficacy'], df_cr.loc[valid_all, 'epsilon'].values.squeeze())
fill(f.climate_configs['gamma_autocorrelation'], df_cr.loc[valid_all, 'gamma'].values.squeeze())
fill(f.climate_configs['sigma_eta'], df_cr.loc[valid_all, 'sigma_eta'].values.squeeze())
fill(f.climate_configs['sigma_xi'], df_cr.loc[valid_all, 'sigma_xi'].values.squeeze())
fill(f.climate_configs['seed'], seed)
fill(f.climate_configs['stochastic_run'], True)
fill(f.climate_configs['use_seed'], True)
fill(f.climate_configs['forcing_4co2'], 2 * erf_2co2 * (1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[valid_all,'F_4xCO2'])/calibrated_f4co2_mean))

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs['iirf_0'], df_cc.loc[valid_all, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], df_cc.loc[valid_all, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], df_cc.loc[valid_all, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], df_cc.loc[valid_all, 'rT'].values.squeeze(), specie='CO2')

# aerosol indirect
fill(f.species_configs['aci_scale'], df_aci.loc[valid_all, 'beta'].values.squeeze())
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_so2'].values.squeeze(), specie='Sulfur')
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_bc'].values.squeeze(), specie='BC')
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_oc'].values.squeeze(), specie='OC')

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# aerosol direct
for specie in df_ari:
    fill(f.species_configs['erfari_radiative_efficiency'], df_ari.loc[valid_all, specie], specie=specie)

# forcing
for specie in df_scaling:
    if specie in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2', 'Volcanic']:
        continue
    fill(f.species_configs['forcing_scale'], df_scaling.loc[valid_all, specie].values.squeeze(), specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], df_scaling.loc[valid_all, 'minorGHG'].values.squeeze(), specie=specie)
fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[valid_all,'F_4xCO2'].values)/calibrated_f4co2_mean, specie='CO2')

# ozone
for specie in df_ozone:
    fill(f.species_configs['ozone_radiative_efficiency'], df_ozone.loc[valid_all, specie], specie=specie)

# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], df_1750co2.loc[valid_all, 'co2_concentration'].values.squeeze(), specie='CO2')

# initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run()

In [ ]:
f.species_configs

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.min(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1), 
        np.max(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 5, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1750.5, 2301), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 16, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1750.5, 2301), 
        np.median(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1750,2300)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios[i])
pl.suptitle('Temperature anomaly')
fig.tight_layout()
pl.savefig('../plots/final_ssp_temperatures.png')

# Temperature diffs w.r.t. 1995-2014

Future periods are 2021-2040, 2041-2060, 2081-2100. Values are 5th, 50th, 95th percentile.

In [ ]:
# ssp119
print(np.percentile(f.temperature[271:291, 0, :, 0].mean(axis=0)-f.temperature[245:265, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 0, :, 0].mean(axis=0)-f.temperature[245:265, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 0, :, 0].mean(axis=0)-f.temperature[245:265, 0, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp126
print(np.percentile(f.temperature[271:291, 1, :, 0].mean(axis=0)-f.temperature[245:265, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 1, :, 0].mean(axis=0)-f.temperature[245:265, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 1, :, 0].mean(axis=0)-f.temperature[245:265, 1, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp245
print(np.percentile(f.temperature[271:291, 2, :, 0].mean(axis=0)-f.temperature[245:265, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 2, :, 0].mean(axis=0)-f.temperature[245:265, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 2, :, 0].mean(axis=0)-f.temperature[245:265, 2, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp370
print(np.percentile(f.temperature[271:291, 3, :, 0].mean(axis=0)-f.temperature[245:265, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 3, :, 0].mean(axis=0)-f.temperature[245:265, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 3, :, 0].mean(axis=0)-f.temperature[245:265, 3, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp585
print(np.percentile(f.temperature[271:291, 7, :, 0].mean(axis=0)-f.temperature[245:265, 7, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 7, :, 0].mean(axis=0)-f.temperature[245:265, 7, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 7, :, 0].mean(axis=0)-f.temperature[245:265, 7, :, 0].mean(axis=0), (5, 50, 95)))

# Temperature diffs w.r.t. 1850-1900

In [ ]:
# ssp119
print(np.percentile(f.temperature[271:291, 0, :, 0].mean(axis=0)-f.temperature[100:151, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 0, :, 0].mean(axis=0)-f.temperature[100:151, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 0, :, 0].mean(axis=0)-f.temperature[100:151, 0, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp126
print(np.percentile(f.temperature[271:291, 1, :, 0].mean(axis=0)-f.temperature[100:151, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 1, :, 0].mean(axis=0)-f.temperature[100:151, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 1, :, 0].mean(axis=0)-f.temperature[100:151, 1, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp245
print(np.percentile(f.temperature[271:291, 2, :, 0].mean(axis=0)-f.temperature[100:151, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 2, :, 0].mean(axis=0)-f.temperature[100:151, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 2, :, 0].mean(axis=0)-f.temperature[100:151, 2, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp370
print(np.percentile(f.temperature[271:291, 3, :, 0].mean(axis=0)-f.temperature[100:151, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 3, :, 0].mean(axis=0)-f.temperature[100:151, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 3, :, 0].mean(axis=0)-f.temperature[100:151, 3, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# ssp585
print(np.percentile(f.temperature[271:291, 7, :, 0].mean(axis=0)-f.temperature[100:151, 7, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[291:311, 7, :, 0].mean(axis=0)-f.temperature[100:151, 7, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[331:351, 7, :, 0].mean(axis=0)-f.temperature[100:151, 7, :, 0].mean(axis=0), (5, 50, 95)))

## Today

In [ ]:
print(np.percentile(f.temperature[245:265, 2, :, 0].mean(axis=0)-f.temperature[100:151, 2, :, 0].mean(axis=0), (5, 50, 95)))

## Forcing

### Methane: SSP245 in 2019

In [ ]:
np.percentile(f.forcing[269, 2, :, 3], (5, 50, 95))

In [ ]:
np.percentile(f.concentration[0, 2, :, 3], (5, 50, 95))
np.percentile(f.concentration[269, 2, :, 3], (5, 50, 95))

In [ ]:
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
conc_ch4 = {}
for scenario in scenarios:
    conc_ch4[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
pl.plot(np.percentile(f.concentration[:, 2, :, 3], (50), axis=1))
pl.plot(conc_ch4['ssp245'][:], color='k')
#pl.xlim(200,225)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 2:5].sum(axis=2) + 
        f.forcing[:351, 2, :, 11:51].sum(axis=2),
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 2:5].sum(axis=2) + 
        f.forcing[:351, 2, :, 11:51].sum(axis=2),
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 2:5].sum(axis=2) + 
        f.forcing[:351, 2, :, 11:51].sum(axis=2),
        axis=1
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 56:58].sum(axis=2),
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 56:58].sum(axis=2),
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 56:58].sum(axis=2),
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 2],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 2],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 2],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 54],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 54],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 54],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 58],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 58],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 58],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 59],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 59],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 59],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 60],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 60],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 60],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 61],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 61],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 61],
        axis=1,
    ), color='k'
)

In [ ]:
pl.fill_between(
    np.arange(1750, 2101),
    np.percentile(
        f.forcing[:351, 2, :, 62],
        5,
        axis=1
    ),
    np.percentile(
        f.forcing[:351, 2, :, 62],
        95,
        axis=1
    ),
    color='k', alpha=0.3
)
pl.plot(
    np.arange(1750, 2101),
    np.median(
        f.forcing[:351, 2, :, 62],
        axis=1,
    ), color='k'
)

In [ ]:
pl.plot(
    np.arange(1750.5, 2301), 
    f.temperature[:, 2, :5, 0]-f.temperature[100:151, 2, :5, 0].mean(axis=0), 
)

## Dump out

In [ ]:
for scen_idx in [('ssp126', 1), ('ssp245', 2), ('ssp370', 3)]:
    df_dump = pd.DataFrame(
        f.temperature[:, scen_idx[1], :, 0]-f.temperature[100:151, scen_idx[1], :, 0].mean(axis=0),
        index = f.timebounds,
        columns = valid_all,
    )
    df_dump.index.rename('year', inplace=True)
    df_dump.to_csv(f'../data/output/temperature_full_ens_{scen_idx[0]}.csv')

## Animate

In [ ]:
%matplotlib notebook

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
def animate(i):
    ax.plot(
        np.arange(1750.5, 2301), 
        f.temperature[:, 2, i, 0]-f.temperature[100:151, 2, i, 0].mean(axis=0), 
    )
    ax.set_xlim(1850, 2100)
    ax.set_ylim(-1, 4)

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
fig, ax = pl.subplots()

ax.axhline(0, ls=':', color='k', zorder=7)
ax.plot(np.arange(1850.5, 2021), gmst, color='k', label='Observations', zorder=10)
ax.legend()
ax.set_title('Calibrated, constrained SSP2-4.5 projections')
ax.set_ylabel('°C relative to 1850-1900')

ani = FuncAnimation(fig, animate, frames=101, interval=200, repeat=False)
ani.save('../plots/ssp245.mp4')

## Save out land use forcing

In [ ]:
f.forcing[:, :, :, 62].shape

In [ ]:
np.save('../data/ar6_ensemble_batches/land-use-erf-from-emissions-driven-ssps.npy', f.forcing[:, :, :, 62])

In [ ]:
df_cr.loc[valid_all, :], df_cc.loc[valid_all, :], df_ari.loc[valid_all, :], df_aci.loc[valid_all, :], df_ozone.loc[valid_all, :], df_scaling.loc[valid_all, :], df_1750co2.loc[valid_all, :]

In [ ]:
params_out = pd.concat((df_cr.loc[valid_all, :], df_cc.loc[valid_all, :], df_ari.loc[valid_all, :], df_aci.loc[valid_all, :], df_ozone.loc[valid_all, :], df_scaling.loc[valid_all, :], df_1750co2.loc[valid_all, :]), axis=1)

In [ ]:
params_out.columns

In [ ]:
params_out.columns = ['gamma', 'c1', 'c2', 'c3', 'kappa1', 'kappa2', 'kappa3', 'epsilon',
       'sigma_eta', 'sigma_xi', 'F_4xCO2', 'r0', 'rU', 'rT', 'rA', 'ari BC', 'ari CH4',
       'ari CO', 'ari N2O', 'ari NH3', 'ari NOx', 'ari OC', 'ari Sulfur', 'ari VOC',
       'ari Equivalent effective stratospheric chlorine', 'shape_so2',
       'shape_bc', 'shape_oc', 'beta', 'o3 CH4', 'o3 N2O',
       'o3 Equivalent effective stratospheric chlorine', 'o3 CO', 'o3 VOC', 'o3 NOx',
       'scale CO2', 'scale CH4', 'scale N2O', 'scale minorGHG', 'scale Stratospheric water vapour',
       'scale Contrails', 'scale Light absorbing particles on snow and ice', 'scale Land use',
       'scale Volcanic', 'solar_amplitude', 'solar_trend', 'co2_concentration_1750']

In [ ]:
params_out.drop(columns=['scale CO2', 'ari CO'], inplace=True)

In [ ]:
params_out.to_csv('../data/output/ar6_calibration_ebm3.csv')

## Save out percentiles as binary

In [ ]:
emissions_species = f.emissions.dropna(dim="specie").specie.data

In [ ]:
concentration_species = f.concentration.dropna(dim="specie").specie.data

In [ ]:
forcing_species = f.forcing.dropna(dim="specie").specie.data

In [ ]:
emissions_species

In [ ]:
emissions_units = [f'kt {specie}/yr' for specie in emissions_species]
emissions_units[0] = 'Gt CO2/yr'
emissions_units[1] = 'Gt CO2/yr'
emissions_units[2] = 'Gt CO2/yr'
emissions_units[3] = 'Mt CH4/yr'
emissions_units[4] = 'Mt N2O/yr'
emissions_units[5] = 'Mt SO2/yr'
emissions_units[6] = 'Mt BC/yr'
emissions_units[7] = 'Mt OC/yr'
emissions_units[8] = 'Mt NH3/yr'
emissions_units[9] = 'Mt NO2/yr'
emissions_units[10] = 'Mt VOC/yr'
emissions_units[11] = 'Mt CO/yr'
emissions_units[53] = 'Mt NO2/yr'
emissions_units

In [ ]:
concentration_species

In [ ]:
concentration_units = ['ppt' for specie in concentration_species]
concentration_units[0] = 'ppm'
concentration_units[1] = 'ppb'
concentration_units[2] = 'ppb'
concentration_units

In [ ]:
xr.Dataset(
    data_vars = dict(
        concentration = (
            ['timebounds', 'scenario', 'percentile', 'specie'], 
            np.percentile(f.concentration.dropna(dim="specie"), (5, 50, 95), axis=2).transpose((1,2,0,3))
        ),
        units = (['specie'], concentration_units)
    ),
    coords = {
        'timebounds': f.timebounds,
        'scenario': f.scenarios,
        'percentile': [5, 50, 95],
        'specie': concentration_species
    }
).to_netcdf('../data/output/ssp_concentration.nc')

In [ ]:
xr.Dataset(
    data_vars = dict(
        forcing = (
            ['timebounds', 'scenario', 'percentile', 'specie'], 
            np.percentile(f.forcing.dropna(dim="specie"), (5, 50, 95), axis=2).transpose((1,2,0,3))
        ),
        units = (['specie'], ['W/m2']*len(forcing_species))
    ),
    coords = {
        'timebounds': f.timebounds,
        'scenario': f.scenarios,
        'percentile': [5, 50, 95],
        'specie': forcing_species
    }
).to_netcdf('../data/output/ssp_forcing.nc')

In [ ]:
xr.Dataset(
    data_vars = dict(
        emissions = (
            ['timepoints', 'scenario', 'specie'], 
            f.emissions[:,:,0,:].dropna(dim="specie").data
        ),
        units = (['specie'], emissions_units)
    ),
    coords = {
        'timepoints': f.timepoints,
        'scenario': f.scenarios,
        'specie': emissions_species
    }
).to_netcdf('../data/output/ssp_emissions.nc')

In [ ]:
xr.Dataset(
    data_vars = dict(
        forcing = (
            ['timebounds', 'scenario', 'percentile'], 
            np.percentile(f.forcing_sum, (5, 50, 95), axis=2).transpose((1,2,0))
        )
    ),
    coords = {
        'timebounds': f.timebounds,
        'scenario': f.scenarios,
        'percentile': [5, 50, 95],
    }
).to_netcdf('../data/output/ssp_forcing_sum.nc')

In [ ]:
xr.Dataset(
    data_vars = dict(
        temperature = (
            ['timebounds', 'scenario', 'percentile'], 
            np.percentile(f.temperature[:, :, :, 0] - f.temperature[100:151, :, :, 0].mean(axis=0), (5, 50, 95), axis=2).transpose((1,2,0))
        )
    ),
    coords = {
        'timebounds': f.timebounds,
        'scenario': f.scenarios,
        'percentile': [5, 50, 95],
    }
).to_netcdf('../data/output/ssp_temperature.nc')

In [ ]:
#f.temperature[:, :, :, 0].to_netcdf('../data/output/ssp_temperature.nc')

In [ ]:
f.concentration[:, :, 0, 2]